# 단어 Level로 번역기 업그레이드하기

In [1]:
import tensorflow

print(tensorflow.__version__)

2.6.0


In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import re
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.head(5)

전체 샘플의 수 : 197463


,eng,fra,cc
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


데이터를 33000개만 사용하고, 그 중 3000개는 테스트용, 30000개는 학습용으로 만들 예정  
또한 원본 데이터 중 필요없는 컬럼 cc 부분을 지우자

In [3]:
lines = lines[['eng', 'fra']][:33000]
lines.head(5)

,eng,fra
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [4]:
lines['eng'][23005]

'Does truth matter?'

### ?와 같은 특수문자는 단어에 포함되지 않도록 단어와 분리시켜야 한다.

In [5]:
for i,v in enumerate(lines['eng']):
    punctuation = re.sub(r'[?]', ' ?', v)
    punctuation = re.sub(r'[!]', ' !', punctuation)
    punctuation = re.sub(r'[.]', ' .', punctuation)
    punctuation = re.sub(r"[']", " ' ", punctuation)
    punctuation = punctuation.lower()
    lines['eng'][i] = punctuation

In [6]:
lines['eng'][23005]

'does truth matter ?'

In [7]:
lines.sample(10)

,eng,fra
27329,we ' re really good .,Nous sommes vraiment bonnes.
29868,i hope we find tom .,J'espère que nous trouverons Tom.
27803,you did your best .,Tu as fais de ton mieux.
22486,you ' re very good .,Tu es fort bonne.
5546,it ' s so easy .,C'est tellement facile.
18699,i hate surprises .,Je déteste les surprises.
24342,i really need you .,J'ai vraiment besoin de toi.
19012,i own a computer .,Je possède un ordinateur.
23931,i hate that movie .,Je déteste ce film.
14264,i like your hat .,J'aime votre chapeau.


In [8]:
print(lines['fra'][23005])

Est-ce que la vérité importe ?


In [9]:
for i,v in enumerate(lines['fra']):
    punctuation = re.sub(r'[?]', ' ?', v)
    punctuation = re.sub(r'[!]', ' !', punctuation)
    punctuation = re.sub(r'[.]', ' .', punctuation)
    punctuation = re.sub(r"[']", " ' ", punctuation)
    punctuation = re.sub(r'[-]', ' - ', punctuation)
    punctuation = punctuation.lower()
    lines['fra'][i] = punctuation

In [10]:
print(lines['fra'][23005])

est - ce que la vérité importe  ?


In [11]:
lines.sample(10)

,eng,fra
29343,"hey, don ' t do that !","hé, ne faites pas ça !"
3232,i nailed it .,j ' ai mis la main dessus .
5845,take control .,prends les commandes .
24685,i ' m a sagittarius .,je suis sagittaire .
18099,he heard a shout .,il entendit un cri .
28547,did you write that ?,as - tu écrit ça ?
19996,may i open a can ?,puis - je ouvrir une cannette ?
25488,no one understood .,personne ne comprit .
18965,i need a kleenex .,il me faut un kleenex .
17940,give me my drink .,donne - moi mon verre !


정규표현식 참고 링크
https://rfriend.tistory.com/748

### 정제된 문장들을 토큰화
공백을 기준으로 단어들을 잘라서 토큰화

In [12]:
eng_token = Tokenizer(char_level=False)
eng_token.fit_on_texts(lines.eng)
input_text = eng_token.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장

딕셔너리로 구성된 단어:인덱스 확인하기

In [13]:
print(eng_token.word_index)

{"'": 1, 'i': 2, 'you': 3, 'tom': 4, 'it': 5, 'is': 6, 's': 7, 'a': 8, 're': 9, 'm': 10, 'we': 11, 't': 12, 'the': 13, 'that': 14, 'he': 15, 'was': 16, 'me': 17, 'this': 18, 'they': 19, 'are': 20, 'to': 21, 'do': 22, 'can': 23, 'my': 24, 'your': 25, 'go': 26, 'have': 27, 'not': 28, 'don': 29, 'll': 30, 'be': 31, 'she': 32, 'no': 33, 'did': 34, 'what': 35, 'all': 36, 'let': 37, 'here': 38, 'like': 39, 'in': 40, 'get': 41, 'up': 42, 'how': 43, 'very': 44, 'on': 45, 'need': 46, 'one': 47, 'want': 48, 'him': 49, 'out': 50, 'love': 51, 'come': 52, 'who': 53, 'please': 54, 'us': 55, 'there': 56, 'just': 57, 'know': 58, 'now': 59, 'stop': 60, 'so': 61, 'of': 62, 'got': 63, 'too': 64, 'help': 65, 'look': 66, 'take': 67, 'good': 68, 'for': 69, 'has': 70, 'at': 71, 'see': 72, 'will': 73, 'were': 74, 've': 75, 'keep': 76, 'had': 77, 'am': 78, 'back': 79, 'give': 80, 'feel': 81, 'saw': 82, 'try': 83, 'stay': 84, 'must': 85, 'happy': 86, 'where': 87, 'home': 88, 'alone': 89, 'didn': 90, 'car': 91, 

각 단어가 몇번 등장하였는지 확인해보고, 등장 빈도가 높은 순서대로 인덱스가 부여되는걸 알 수 있다.

In [14]:
print(eng_token.word_counts)

OrderedDict([('go', 815), ('hi', 14), ('run', 71), ('who', 319), ('wow', 5), ('duck', 4), ('fire', 17), ('help', 275), ('hide', 10), ('jump', 16), ('stop', 290), ('wait', 119), ('begin', 13), ('on', 426), ('hello', 16), ('i', 8791), ('see', 232), ('try', 187), ('won', 162), ('oh', 12), ('no', 627), ('relax', 23), ('smile', 17), ('sorry', 46), ('attack', 14), ('buy', 53), ('it', 3408), ('cheers', 4), ('eat', 144), ('get', 462), ('up', 445), ('now', 292), ('got', 279), ('hop', 2), ('in', 490), ('hug', 16), ('me', 1245), ('fell', 42), ('fled', 1), ('knit', 4), ('know', 296), ('left', 100), ('lied', 27), ('lost', 150), ('paid', 32), ('pass', 10), ('quit', 53), ("'", 9487), ('m', 1837), ('19', 2), ('ok', 77), ('listen', 41), ('way', 83), ('really', 97), ('thanks', 44), ('we', 1786), ('ask', 65), ('tom', 3657), ('him', 357), ('awesome', 31), ('be', 651), ('calm', 46), ('cool', 47), ('fair', 25), ('kind', 38), ('nice', 121), ('beat', 33), ('burn', 8), ('bury', 5), ('call', 99), ('us', 300), (

In [15]:
print(lines.eng[23005])
input_text[23005]

does truth matter ?


[131, 802, 833]

프랑스어 문장은 문장 시작 부분에 \t를 끝 부분에 \n을 넣어 구분을 해준다.

In [16]:
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '<sos> '+ x + ' <eos>')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
16952,where ' s the loo ?,<sos> où sont les chiottes ? <eos>
19240,i won ' t be quiet .,<sos> je ne serai pas tranquille . <eos>
6876,drink it down .,<sos> buvez - le ! <eos>
4963,i have a cat .,<sos> j ' ai un chat . <eos>
24296,i owe you a lunch .,<sos> je vous dois un déjeuner . <eos>


In [17]:
fra_token = Tokenizer(char_level=False)
fra_token.fit_on_texts(lines.fra)
target_text = fra_token.texts_to_sequences(lines.fra)    # 단어를 숫자값 인덱스로 변환하여 저장
target_text[23005]

[1, 5, 25, 31, 22, 632, 445, 2]

In [18]:
print(fra_token.word_index)

{'sos': 1, 'eos': 2, "'": 3, 'je': 4, 'est': 5, 'tom': 6, 'vous': 7, 'pas': 8, 'j': 9, 'il': 10, 'le': 11, 'ai': 12, 'de': 13, 'nous': 14, 'a': 15, 'ne': 16, 'suis': 17, 'c': 18, 'tu': 19, 'l': 20, 'un': 21, 'la': 22, 'en': 23, 'à': 24, 'ce': 25, 'n': 26, 'me': 27, 'êtes': 28, 'une': 29, 'ça': 30, 'que': 31, 'les': 32, 'moi': 33, 'es': 34, 'elle': 35, 'd': 36, 's': 37, 'sont': 38, 'ils': 39, 'y': 40, 'était': 41, 'sommes': 42, 't': 43, 'm': 44, 'fait': 45, 'tout': 46, 'elles': 47, 'qui': 48, 'des': 49, 'mon': 50, 'aime': 51, 'te': 52, 'toi': 53, 'as': 54, 'bien': 55, 'très': 56, 'été': 57, 'se': 58, 'du': 59, 'ici': 60, 'besoin': 61, 'air': 62, 'faire': 63, 'avez': 64, 'peux': 65, 'ont': 66, 'personne': 67, 'avons': 68, 'veux': 69, 'votre': 70, 'va': 71, 'qu': 72, 'cela': 73, 'on': 74, 'fais': 75, 'étais': 76, 'ton': 77, 'faut': 78, 'là': 79, 'comment': 80, 'tous': 81, 'ma': 82, 'aller': 83, 'trop': 84, 'monde': 85, 'au': 86, 'adore': 87, 'pour': 88, 'train': 89, 'être': 90, 'maintenan

In [19]:
print(fra_token.word_counts)

OrderedDict([('sos', 33000), ('va', 285), ('eos', 33000), ('marche', 38), ('en', 1483), ('route', 26), ('bouge', 16), ('salut', 15), ('cours\u202f', 2), ('courez\u202f', 2), ('prenez', 87), ('vos', 53), ('jambes', 11), ('à', 1414), ('cous', 3), ('file', 7), ('filez', 4), ('cours', 17), ('fuyez', 3), ('fuyons', 2), ('qui', 470), ('ça', 920), ('alors\u202f', 1), ('waouh\xa0', 1), ('wah\xa0', 1), ('terre\xa0', 1), ('baisse', 8), ('toi\xa0', 7), ('baissez', 2), ('vous\xa0', 11), ('au', 218), ('feu', 27), ('l', 1665), ("'", 10350), ('aide\u202f', 4), ('cache', 6), ('toi', 378), ('cachez', 1), ('vous', 2992), ('saute', 7), ('suffit\u202f', 2), ('stop\u202f', 1), ('arrête', 127), ('attends', 37), ('attendez', 32), ('commencez', 10), ('commence', 17), ('poursuis', 6), ('continuez', 37), ('poursuivez', 3), ('bonjour', 17), ('je', 5857), ('comprends', 18), ('aha', 2), ('j', 2628), ('essaye', 21), ('ai', 2211), ('gagné', 46), ('emporté', 12), ('j’ai', 80), ('oh', 8), ('non', 76), ('calme', 62), (

In [20]:
print(lines.fra[23005])
target_text[23005]

<sos> est - ce que la vérité importe  ? <eos>


[1, 5, 25, 31, 22, 632, 445, 2]

In [21]:
eng_vocab_size = len(eng_token.word_index) + 1
fra_vocab_size = len(fra_token.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4702
프랑스어 단어장의 크기 : 9455


In [22]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 9
프랑스어 시퀀스의 최대 길이 17


In [23]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4702
프랑스어 단어장의 크기 : 9455
영어 시퀀스의 최대 길이 9
프랑스어 시퀀스의 최대 길이 17


In [24]:
print(lines.eng[:3])
print(input_text[:3])
print(lines.fra[:3])
print(target_text[:3])

0    go .
1    go .
2    go .
Name: eng, dtype: object
[[26], [26], [26]]
0          <sos> va  ! <eos>
1       <sos> marche . <eos>
2    <sos> en route  ! <eos>
Name: fra, dtype: object
[[1, 71, 2], [1, 340, 2], [1, 23, 481, 2]]


In [25]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_token.word_index['eos'] ]for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_token.word_index['sos'] ]for line in target_text]

print(decoder_input[:3])
print(decoder_target[:3])

[[1, 71], [1, 340], [1, 23, 481]]
[[71, 2], [340, 2], [23, 481, 2]]


In [26]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 9)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [28]:
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_target[:3])

[[26  0  0  0  0  0  0  0  0]
 [26  0  0  0  0  0  0  0  0]
 [26  0  0  0  0  0  0  0  0]]
[[  1  71   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1 340   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1  23 481   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
[[ 71   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [340   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 23 481   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
